In [4]:
import numpy as np 
import torch
from utility_funcs import get_train_labels_test
from scipy import stats
from torch.utils.data import Dataset, DataLoader


from sklearn.preprocessing import StandardScaler

In [5]:
!nvidia-smi

Wed Apr 13 18:39:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8     4W /  N/A |      0MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
train, labels, test = get_train_labels_test()

In [ ]:
train.head()

: 

In [ ]:
labels.head()

: 

In [ ]:
test.head()

: 

In [ ]:
train.describe().round(2)

: 

### Data prep

In [ ]:
train["sequence_in_file"] = train['sequence'] - train['sequence'].min()
test["sequence_in_file"] = test['sequence'] - test['sequence'].min()

ctd = ['sequence', 'subject', 'step']
train.drop(ctd, inplace = True)
test.drop(ctd, inplace = True)

In [ ]:
s=StandardScaler()

X_train = train.drop('state', axis=1).values
X_train = s.fit_transform(X_train)
y = train['state'].values

X_test = s.transform(test)

: 

In [1]:
class DatasetGenerator(Dataset):
    """ inherit from the dataset class, pytorch 
    """


    def __init__(self,time_series, labels = None):
        super().__init__()
        self.time_series = time_series
        self.labels = labels
        self.cols = [col for col in self.time_series if 'sensor' in col]

    def __len__(self):
        if not self.labels:
            return len(self.time_series) // 60 
        return len(self.labels)

    def __getitem__(self, index):
        if not self.labels:
            label_tensor = np.nan
        else: 
            label = self.labels[self.labels.sequence_local == index].state.iloc[0]
            label_tensor = torch.tensor(label, dtype=torch.long)

        ts_df = self.time_series[self.time_series.sequence_local == index][self.cols].to_numpy()
        sensor_array = torch.tensor(ts_df, dtype=torch.float32)

        return sensor_array, label_tensor

NameError: name 'Dataset' is not defined